In [5]:
import cv2
import torch
import numpy as np
import os
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image
import tensorflow as tf




ImportError: cannot import name 'GradScaler' from 'torch.amp' (c:\Users\Loc\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\amp\__init__.py)

In [ ]:
# Thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model nhận diện khuôn mặt 
model = tf.keras.models.load_model("mobilenetv2.h5")

# Load mô hình MTCNN và FaceNet
mtcnn = MTCNN(image_size=160, margin=20, device=device)
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Load danh sách nhãn (theo thứ tự đã dùng khi huấn luyện model)
embedding_data = np.load("data_split/train_embeddings.npz")  # hoặc file tổng hợp chứa toàn bộ embedding
class_names = sorted(np.unique(embedding_data["labels"]))

# Mở webcam
cap = cv2.VideoCapture(0)
print("[INFO] Webcam đang mở. Nhấn 'q' để thoát.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize nhỏ cho nhanh (nếu cần), chuyển sang RGB cho MTCNN
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(rgb_frame)

    # Dò khuôn mặt
    boxes, _ = mtcnn.detect(img_pil)

    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = [int(b) for b in box]
            face = frame[y1:y2, x1:x2]
            if face.size == 0:
                continue
            try:
                face_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
                face_tensor = mtcnn(face_pil)
                if face_tensor is None:
                    continue
                face_tensor = face_tensor.unsqueeze(0).to(device)

                with torch.no_grad():
                    embedding = facenet(face_tensor).cpu().numpy()

                # Dự đoán class
                pred = model.predict(embedding)
                pred_index = np.argmax(pred)
                confidence = pred[0][pred_index]

                label = f"{class_names[pred_index]} ({confidence*100:.2f}%)"

                # Hiển thị
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
            except Exception as e:
                print("Lỗi xử lý khuôn mặt:", e)
                continue

    cv2.imshow("Face Recognition (FaceNet + MobileNetV2)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[INFO] Webcam đang mở. Nhấn 'q' để thoát.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1